In [21]:
# !pip uninstall virny -y

In [22]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [23]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

## Import dependencies

In [25]:
import os
import pandas as pd
from pprint import pprint
from datetime import datetime, timezone

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from virny.utils.custom_initializers import create_config_obj, read_model_metric_dfs, create_models_config_from_tuned_params_df
from virny.user_interfaces.metrics_computation_interfaces import compute_metrics_multiple_runs_with_db_writer
from virny.preprocessing.basic_preprocessing import preprocess_dataset
from virny.custom_classes.metrics_composer import MetricsComposer
from virny.utils.model_tuning_utils import tune_ML_models
from virny.datasets import CompasWithoutSensitiveAttrsDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_compas_models_params_for_tuning

## Define Input Variables

In [26]:
DATASET_SPLIT_SEED = EXPERIMENT_SEEDS[0]
MODELS_TUNING_SEED = EXPERIMENT_SEEDS[0]

In [27]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
# ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'error_analysis'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'compas_config.yaml')
config = create_config_obj(config_yaml_path=config_yaml_path)

In [28]:
models_params_for_tuning = get_compas_models_params_for_tuning(MODELS_TUNING_SEED)

## Define a db writer and custom fields to insert into your database

In [29]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [30]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
    'preprocessing_techniques': 'cat: mode, num: median',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  026172c9-4cf6-425b-ba25-8cd7660130f6


## Initialize custom objects

In [31]:
data_loader = CompasWithoutSensitiveAttrsDataset()
data_loader.X_data[data_loader.X_data.columns[:5]].head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45
0,0.0,-2.340451,1.0,-15.010999,1
1,0.0,0.000000,0.0,0.000000,1
2,0.0,0.000000,0.0,0.000000,0
3,0.0,0.000000,0.0,6.000000,1
4,0.0,0.000000,0.0,7.513697,1


In [32]:
column_transformer = ColumnTransformer(transformers=[
    ('categorical_features', OneHotEncoder(handle_unknown='ignore', sparse=False), data_loader.categorical_columns),
    ('numerical_features', StandardScaler(), data_loader.numerical_columns),
])

In [33]:
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, TEST_SET_FRACTION, DATASET_SPLIT_SEED)

### Tune models and create a models config for metrics computation

In [34]:
tuned_params_df, models_config = tune_ML_models(models_params_for_tuning, base_flow_dataset, config.dataset_name, n_folds=3)
tuned_params_df

2023/06/22, 01:59:37: Tuning DecisionTreeClassifier...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
2023/06/22, 01:59:39: Tuning for DecisionTreeClassifier is finished [F1 score = 0.6472851634267528, Accuracy = 0.6521464646464646]



,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,0.647285,0.652146,"{'criterion': 'entropy', 'max_depth': 20, 'max..."


In [35]:
now = datetime.now(timezone.utc)
date_time_str = now.strftime("%Y%m%d__%H%M%S")
tuned_df_path = os.path.join(ROOT_DIR, 'results', 'models_tuning', f'tuning_results_{config.dataset_name}_{date_time_str}.csv')
tuned_params_df.to_csv(tuned_df_path, sep=",", columns=tuned_params_df.columns, float_format="%.4f", index=False)

Create models_config from the saved tuned_params_df for higher reliability

In [36]:
models_config = create_models_config_from_tuned_params_df(models_params_for_tuning, tuned_df_path)
pprint(models_config)

{'DecisionTreeClassifier': DecisionTreeClassifier(criterion='entropy', max_depth=20, max_features='sqrt',
                       min_samples_split=0.1, random_state=100)}


## Subgroup Metrics Computation

In [37]:
multiple_run_metrics_dct = compute_metrics_multiple_runs_with_db_writer(base_flow_dataset, config, models_config,
                                                                        custom_table_fields_dct, db_writer_func)

Multiple runs progress:   0%|          | 0/2 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Look at several columns in top rows of computed metrics

In [38]:
sample_model_metrics_df = multiple_run_metrics_dct[list(models_config.keys())[0]]
sample_model_metrics_df[sample_model_metrics_df.columns[:6]].head(20)

,Metric,overall,sex_priv,sex_priv_correct,sex_priv_incorrect,sex_dis
0,Mean,0.536018,0.576907,0.587857,0.546899,0.526347
1,Std,0.066000,0.064240,0.058124,0.081005,0.066416
2,IQR,0.064431,0.062262,0.057667,0.074855,0.064944
3,Aleatoric_Uncertainty,0.869460,0.861114,0.856052,0.874989,0.871434
4,Overall_Uncertainty,0.890969,0.885508,0.875024,0.914242,0.892260
5,Jitter,0.134428,0.097690,0.081381,0.142387,0.143117
6,Per_Sample_Accuracy,0.684470,0.720792,0.952703,0.085185,0.675878
7,Label_Stability,0.828030,0.885149,0.905405,0.829630,0.814520
8,TPR,0.657692,0.600000,1.000000,0.000000,0.668182
9,TNR,0.746269,0.819672,1.000000,0.000000,0.724638


## Group Metrics Composition

In [39]:
models_metrics_dct = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH, model_names=list(models_config.keys()))

In [40]:
metrics_composer = MetricsComposer(models_metrics_dct, config.sensitive_attributes_dct)

Compute composed metrics

In [41]:
models_composed_metrics_df = metrics_composer.compose_metrics()

In [42]:
models_composed_metrics_df.head(20)

,Metric,sex,race,sex&race,Model_Name
0,Equalized_Odds_TPR,0.152273,0.281008,0.372549,DecisionTreeClassifier
1,Equalized_Odds_FPR,0.059832,0.114977,0.116379,DecisionTreeClassifier
2,Equalized_Odds_FNR,-0.152273,-0.281008,-0.372549,DecisionTreeClassifier
3,Disparate_Impact,1.098930,1.360771,1.359477,DecisionTreeClassifier
4,Statistical_Parity_Difference,0.084091,0.267395,0.269608,DecisionTreeClassifier
5,Accuracy_Parity,0.018237,0.056400,0.089813,DecisionTreeClassifier
6,Label_Stability_Ratio,0.955201,0.967051,0.906661,DecisionTreeClassifier
7,IQR_Parity,0.011284,0.009692,0.020932,DecisionTreeClassifier
8,Std_Parity,0.008264,0.005659,0.012393,DecisionTreeClassifier
9,Std_Ratio,1.142134,1.092229,1.221141,DecisionTreeClassifier
